In [2]:
import json
import numpy as np
import os
from tqdm import tqdm

In [3]:
PROMPT_PATH = "../prompts"

In [4]:
datasets = {
    "dtd": [2, 4, 8, 16, 32, 46, 60], 
    "fgvc_aircraft": [2, 4, 8, 14, 20, 30], 
    "sun397": [2, 4, 8, 16, 23, 30], 
    "flowers": [2, 4, 8, 14, 20, 30],
}

n_seeds = 10

In [5]:
for dataset in ["sun397"]:
    with open(f"../prompts/{dataset}/{dataset}_llama3_prompts_full.json", "r") as f:
        prompt_dict = json.load(f)
    with open(f"../prompts/{dataset}/{dataset}_llama3_prompts_full.json", "w") as f:
        json.dump({dataset: prompt_dict}, f, indent=2)

In [6]:
for dataset in ["sun397"]:
    with open(f"../prompts/{dataset}/{dataset}_llama3_prompts_full.json", "r") as f:
        prompt_dict = json.load(f)[dataset]

    for sample_size in datasets[dataset]:
        for seed in tqdm(range(n_seeds)):
            output = {classname.replace("(", "").replace(")", ""): [] for classname in prompt_dict}
            for c, classname in enumerate(prompt_dict):
                idx = np.random.RandomState(seed + c).choice(datasets[dataset][-1], sample_size, replace=False)
                for i in idx:
                    output[classname.replace("(", "").replace(")", "")].append(prompt_dict[classname][i])
            fp = os.path.join(PROMPT_PATH, f"{dataset}/sample_size_{sample_size:02d}_seed_{seed}.json") 
            with open(fp, 'w') as file:
                json.dump({dataset: output}, file, indent=4)

100%|██████████| 10/10 [00:00<00:00, 13.12it/s]
